# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f84d8795220>
├── 'a' --> tensor([[ 0.1657,  0.7004, -0.0305],
│                   [-0.7508,  0.0089, -0.5143]])
└── 'x' --> <FastTreeValue 0x7f84d8795100>
    └── 'c' --> tensor([[ 0.8369,  1.1530, -0.6796,  0.2164],
                        [-1.1411,  0.6350,  2.4495,  0.8238],
                        [ 0.7780,  0.8311,  1.4110,  0.6833]])

In [4]:
t.a

tensor([[ 0.1657,  0.7004, -0.0305],
        [-0.7508,  0.0089, -0.5143]])

In [5]:
%timeit t.a

59.4 ns ± 0.064 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f84d8795220>
├── 'a' --> tensor([[-0.5466, -0.5771,  1.0889],
│                   [-0.9778, -1.0110,  0.4463]])
└── 'x' --> <FastTreeValue 0x7f84d8795100>
    └── 'c' --> tensor([[ 0.8369,  1.1530, -0.6796,  0.2164],
                        [-1.1411,  0.6350,  2.4495,  0.8238],
                        [ 0.7780,  0.8311,  1.4110,  0.6833]])

In [7]:
%timeit t.a = new_value

58.8 ns ± 0.0492 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1657,  0.7004, -0.0305],
               [-0.7508,  0.0089, -0.5143]]),
    x: Batch(
           c: tensor([[ 0.8369,  1.1530, -0.6796,  0.2164],
                      [-1.1411,  0.6350,  2.4495,  0.8238],
                      [ 0.7780,  0.8311,  1.4110,  0.6833]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1657,  0.7004, -0.0305],
        [-0.7508,  0.0089, -0.5143]])

In [11]:
%timeit b.a

56.6 ns ± 0.0483 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7126, -0.5419,  0.8370],
               [-0.3626,  0.8733, -0.1049]]),
    x: Batch(
           c: tensor([[ 0.8369,  1.1530, -0.6796,  0.2164],
                      [-1.1411,  0.6350,  2.4495,  0.8238],
                      [ 0.7780,  0.8311,  1.4110,  0.6833]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 2.47 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

835 ns ± 2.11 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 40.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 315 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8420a51d30>
├── 'a' --> tensor([[[ 0.1657,  0.7004, -0.0305],
│                    [-0.7508,  0.0089, -0.5143]],
│           
│                   [[ 0.1657,  0.7004, -0.0305],
│                    [-0.7508,  0.0089, -0.5143]],
│           
│                   [[ 0.1657,  0.7004, -0.0305],
│                    [-0.7508,  0.0089, -0.5143]],
│           
│                   [[ 0.1657,  0.7004, -0.0305],
│                    [-0.7508,  0.0089, -0.5143]],
│           
│                   [[ 0.1657,  0.7004, -0.0305],
│                    [-0.7508,  0.0089, -0.5143]],
│           
│                   [[ 0.1657,  0.7004, -0.0305],
│                    [-0.7508,  0.0089, -0.5143]],
│           
│                   [[ 0.1657,  0.7004, -0.0305],
│                    [-0.7508,  0.0089, -0.5143]],
│           
│                   [[ 0.1657,  0.7004, -0.0305],
│                    [-0.7508,  0.0089, -0.5143]]])
└── 'x' --> <FastTreeValue 0x7f8420a518e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 40.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f84d880e070>
├── 'a' --> tensor([[ 0.1657,  0.7004, -0.0305],
│                   [-0.7508,  0.0089, -0.5143],
│                   [ 0.1657,  0.7004, -0.0305],
│                   [-0.7508,  0.0089, -0.5143],
│                   [ 0.1657,  0.7004, -0.0305],
│                   [-0.7508,  0.0089, -0.5143],
│                   [ 0.1657,  0.7004, -0.0305],
│                   [-0.7508,  0.0089, -0.5143],
│                   [ 0.1657,  0.7004, -0.0305],
│                   [-0.7508,  0.0089, -0.5143],
│                   [ 0.1657,  0.7004, -0.0305],
│                   [-0.7508,  0.0089, -0.5143],
│                   [ 0.1657,  0.7004, -0.0305],
│                   [-0.7508,  0.0089, -0.5143],
│                   [ 0.1657,  0.7004, -0.0305],
│                   [-0.7508,  0.0089, -0.5143]])
└── 'x' --> <FastTreeValue 0x7f84d880e9a0>
    └── 'c' --> tensor([[ 0.8369,  1.1530, -0.6796,  0.2164],
                        [-1.1411,  0.6350,  2.4495,  0.8238],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 35.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.8 µs ± 349 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.8369,  1.1530, -0.6796,  0.2164],
                       [-1.1411,  0.6350,  2.4495,  0.8238],
                       [ 0.7780,  0.8311,  1.4110,  0.6833]],
              
                      [[ 0.8369,  1.1530, -0.6796,  0.2164],
                       [-1.1411,  0.6350,  2.4495,  0.8238],
                       [ 0.7780,  0.8311,  1.4110,  0.6833]],
              
                      [[ 0.8369,  1.1530, -0.6796,  0.2164],
                       [-1.1411,  0.6350,  2.4495,  0.8238],
                       [ 0.7780,  0.8311,  1.4110,  0.6833]],
              
                      [[ 0.8369,  1.1530, -0.6796,  0.2164],
                       [-1.1411,  0.6350,  2.4495,  0.8238],
                       [ 0.7780,  0.8311,  1.4110,  0.6833]],
              
                      [[ 0.8369,  1.1530, -0.6796,  0.2164],
                       [-1.1411,  0.6350,  2.4495,  0.8238],
                       [ 0.7780,  0.8311,  1.4110,  0.6833]],

In [26]:
%timeit Batch.stack(batches)

77.2 µs ± 180 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.8369,  1.1530, -0.6796,  0.2164],
                      [-1.1411,  0.6350,  2.4495,  0.8238],
                      [ 0.7780,  0.8311,  1.4110,  0.6833],
                      [ 0.8369,  1.1530, -0.6796,  0.2164],
                      [-1.1411,  0.6350,  2.4495,  0.8238],
                      [ 0.7780,  0.8311,  1.4110,  0.6833],
                      [ 0.8369,  1.1530, -0.6796,  0.2164],
                      [-1.1411,  0.6350,  2.4495,  0.8238],
                      [ 0.7780,  0.8311,  1.4110,  0.6833],
                      [ 0.8369,  1.1530, -0.6796,  0.2164],
                      [-1.1411,  0.6350,  2.4495,  0.8238],
                      [ 0.7780,  0.8311,  1.4110,  0.6833],
                      [ 0.8369,  1.1530, -0.6796,  0.2164],
                      [-1.1411,  0.6350,  2.4495,  0.8238],
                      [ 0.7780,  0.8311,  1.4110,  0.6833],
                      [ 0.8369,  1.1530, -0.6796,  0.2164],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 375 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
